In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import re
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [2]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36


/tmp/ipykernel_221066/465125870.py:7: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=options)


In [3]:
driver.get('https://finesthomee.com/categories/317158/%D8%A3%D8%B6%D8%A7%D8%A1%D8%A7%D8%AA-%D9%82%D8%B1%D9%8A%D8%A8%D8%A7')

In [155]:
driver.find_element_by_class_name('product-sku').get_attribute('innerHTML')

'<strong>رمز المنتج: </strong> <span>8022</span>'

In [159]:
sku1 = driver.find_element_by_xpath('//div[@class="product-sku"]/span')
sku = sku1.get_attribute('innerHTML')

'8022'

In [ ]:
('6723,6724,113',	'اضاءات'),
('42,42800'	'مباخر'),
('43816,43,113',	'شمعدان'),
('44100,44101',	'بيالات /فناجيل /اكواب'),

('42915,42916,42922,113','حامل'),
('56,49,113',	'غطاء خداديات '),
('42915,42916',	'أطقم منزلية وتقديمات'),
('42927',	'كوسترات'),



In [4]:
urls = [
    {'url': 'https://finesthomee.com/categories/317158', 'cat':'أضاءات', 'categories': '6723,6724,113',  'count': 2},
    {'url': 'https://finesthomee.com/categories/317157', 'cat':'مباخر', 'categories': '42,42800',  'count': 3},
    {'url': 'https://finesthomee.com/categories/317156', 'cat':'شمعدان', 'categories': '43816,43,113',  'count': 2},
    {'url': 'https://finesthomee.com/categories/163856', 'cat':'أكواب | فناجيل | بيالات', 'categories': '44100,44101',  'count': 3},
    {'url': 'https://finesthomee.com/categories/68206', 'cat':'أطقم منزلية وتقديمات', 'categories': '42915,42916',  'count': 3},
    {'url': 'https://finesthomee.com/categories/68205', 'cat':'كوسترات',  'categories': '42927','count': 2},
    {'url': 'https://finesthomee.com/categories/68203', 'cat':'غطاء خداديات', 'categories': '56,49,113', 'count': 2},


]

df = pd.DataFrame(urls)
df.to_excel('Finesthomee_urls.xlsx')

In [7]:
urls = pd.read_excel('Finesthomee_urls.xlsx')
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/finesthomee/model_urls.xlsx')
list_urls = []
for index, row in urls.iterrows():
    list_urls.append({'url': row['url'], 'count': row['count'], 'cat': row['cat'], 'categories': row['categories']})
for i, url in enumerate(list_urls):
    print('Count: ', i)
    time.sleep(1)
    for i in range(1,url['count']):
        driver.get(url['url']+f'?page={i}')
        products = driver.find_elements_by_class_name('shop-thumbnail')
        len(products)
        list_products = [product.find_element_by_xpath('a').get_attribute('href') for product in products]
        len(list_products)
        for product in list_products:
            data = {
                'url': product,
                'categories1': url['cat'],
                'categories': url['categories'],
            }
            df1 = pd.DataFrame([data])
            df = pd.concat([df, df1], ignore_index=True)
            df.to_excel('Fines_list_urls.xlsx')
# urls = [product.find_elements_by_xpath('a')]

Count:  0
Count:  1
Count:  2
Count:  3
Count:  4
Count:  5
Count:  6


In [105]:
def extraire_ele(name, soup):
    try:
        toto = soup.find('li', text=re.compile(f'^{name}')).text.replace(name, '').replace(':', '').strip()
    except:
        toto = ''
    if toto == '':
        try:
            toto = soup.find('p', text=re.compile(f'^{name}')).text.replace(name, '').replace(':', '').strip()
            
        except:
            try:
                toto = soup.find('li', text=re.compile(f'{name}')).text.replace(name, '').replace(':', '').strip()
            except:
                toto = ''
    return toto

In [116]:
name = driver.find_element_by_class_name('product-title').text.strip()
special_price = ''

price = driver.find_element_by_id('product-old-price').text.replace('ر.س', '').strip()
special_price = driver.find_element_by_id('product-price').text.replace('ر.س', '').strip()


# weight = driver.find_element_by_class_name('product-meta').get_attribute('innerHTMl')
# description = driver.find_element_by_id('description-tab').text
details = driver.find_element_by_id('description-tab').get_attribute('innerHTML')
soup = BeautifulSoup(details, 'html.parser')
# height = extraire_ele('الارتفاع', soup)
# diameter = extraire_ele('القطر', soup)
# lenght = extraire_ele('الطول', soup)
# ('price:', price , '----', 'sp price', special_price)
amper = extraire_ele('سعة البطارية', soup)
power = extraire_ele('الطاقة المقدرة', soup)
rechargeable = extraire_ele('مدة الشحن', soup)
no_of_lamps = extraire_ele('عدد لمبات المصباح', soup)
try:
    with_controller = soup.find('span', text=re.compile('ريموت لتحكم ')).text.strip()
    with_controller  ='Yes'
except:
    with_controller = 'No'
height = extraire_ele('الطول', soup)
height

'4.5'

In [65]:
images_thmbs = driver.find_element_by_class_name('image-thumbs').find_elements_by_tag_name('img')
print(len(images_thmbs))
list_images = []

for img in images_thmbs:
    img.click()
    list_images.append(driver.find_element_by_id('main-img').get_attribute('src'))
    time.sleep(0.3)

3
https://media.zid.store/cdn-cgi/image/f=auto/https://media.zid.store/c11f34bd-67a0-4d98-abe5-df8d247f88d2/178f1889-3621-43b7-8b52-fcc7a8152bbb.jpg
https://media.zid.store/cdn-cgi/image/f=auto/https://media.zid.store/c11f34bd-67a0-4d98-abe5-df8d247f88d2/2488d920-ec4b-4724-a020-d54274869022.jpg
https://media.zid.store/cdn-cgi/image/f=auto/https://media.zid.store/c11f34bd-67a0-4d98-abe5-df8d247f88d2/13bf1324-ab24-4af6-ae5e-2e2990f4fbd4.jpg


In [81]:


def extract_colors(detail):
    details = detail.split(' ')
    for i, color in enumerate(details):
        if 'لون' in color:
            return (details[i+1])

In [ ]:
co

In [4]:
r = requests.get('https://finesthomee.com/categories/317158/%D8%A3%D8%B6%D8%A7%D8%A1%D8%A7%D8%AA-%D9%82%D8%B1%D9%8A%D8%A8%D8%A7')
soup = BeautifulSoup(r.text, 'html.parser')
# print(soup.prettify())
links = soup.find_all('div', {'class': 'zid-item product'})
len(links)
# name = soup.find('h1', {'class': 'productname'}).text.strip()
# sku = soup.find('span', {'class': 'productname_small pb-3'}).find('p').text.strip()
# price = soup.find('h3', {'class': 'productprice pb-3'}).find('span').text.strip()
# description = soup.find('div', {'id': 'product_details'}).text
# details = soup.find('div', {'id': 'product_details'})
# images = soup.find('ul', {'class': 'slides'}).find_all('li')
# list_images = [img.find('img')['src'] for img in images]


0

In [141]:
df = pd.read_excel('/home/wafistos/Documents/Projects/finesthomee/-finesthomee_update_01_02.xlsx.xlsx')

In [167]:
list_cate = [
('6723,6724,113',	'اضاءات'),
('42,42800'	'مباخر'),
('43816,43,113',	'شمعدان'),
('44100,44101',	'بيالات /فناجيل /اكواب'),

('42915,42916,42922,113','حامل'),
('56,49,113',	'غطاء خداديات '),
('42915,42916',	'أطقم منزلية وتقديمات'),
('42927',	'كوسترات'),



]

In [139]:
count = 2000
for index, row in df.iterrows():
    row['sku'] = str(2000 + index)

In [142]:
df['sku']

0      FIN-0      2001\n1      2001\n2      2001\n3  ...
1      FIN-0      2001\n1      2001\n2      2001\n3  ...
2      FIN-0      2001\n1      2001\n2      2001\n3  ...
3      FIN-0      2001\n1      2001\n2      2001\n3  ...
4      FIN-0      2001\n1      2001\n2      2001\n3  ...
                             ...                        
162    FIN-0      2001\n1      2001\n2      2001\n3  ...
163    FIN-0      2001\n1      2001\n2      2001\n3  ...
164    FIN-0      2001\n1      2001\n2      2001\n3  ...
165    FIN-0      2001\n1      2001\n2      2001\n3  ...
166    FIN-0      2001\n1      2001\n2      2001\n3  ...
Name: sku, Length: 167, dtype: object

In [124]:
df.loc[df['price'].isnull(), 'price'] = df['special_price']
df.loc[df['price'] == df['special_price'], 'special_price'] = '__EMPTY__VALUE__'

In [ ]:
for cat in list_cat:
    df.loc[df['categories']]